[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/00-introduction.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/00-introduction.ipynb)

# Semantic Router Intro

The Semantic Router library can be used as a super fast route making layer on top of LLMs. That means rather than waiting on a slow agent to decide what to do, we can use the magic of semantic vector space to make routes. Cutting route making time down from seconds to milliseconds.

## Getting Started

We start by installing the library:

In [1]:
!pip install -qU semantic-router


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


We start by defining a dictionary mapping routes to example phrases that should trigger those routes.

In [3]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

d:\Program_Installation\anaconda\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's define another for good measure:

In [4]:
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

Now we initialize our embedding model:

In [5]:
import os
from getpass import getpass
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
#     "Enter Cohere API Key: "
# )
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

# encoder = CohereEncoder()
encoder = OpenAIEncoder()

Now we define the `RouteLayer`. When called, the route layer will consume text (a query) and output the category (`Route`) it belongs to — to initialize a `RouteLayer` we need our `encoder` model and a list of `routes`.

In [6]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes)

2024-05-02 12:09:30 INFO semantic_router.utils.logger local
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 1 length: 34
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 1 trunc length: 34
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 2 length: 51
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 2 trunc length: 51
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 3 length: 33
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 3 trunc length: 33
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 4 length: 33
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 4 trunc length: 33
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 5 length: 38
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 5 trunc length: 38
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 6 length: 27
2024-05-02 12:09:30 INFO semantic_router.utils.logger Document 6 trunc len

Now we can test it:

In [7]:
rl("don't you love politics?")

2024-05-02 12:09:35 INFO semantic_router.utils.logger Document 1 length: 24
2024-05-02 12:09:35 INFO semantic_router.utils.logger Document 1 trunc length: 24


RouteChoice(name='politics', function_call=None, similarity_score=None)

In [8]:
rl("how's the weather today?")

2024-05-02 12:09:37 INFO semantic_router.utils.logger Document 1 length: 24
2024-05-02 12:09:37 INFO semantic_router.utils.logger Document 1 trunc length: 24


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

Both are classified accurately, what if we send a query that is unrelated to our existing `Route` objects?

In [9]:
rl("I'm interested in learning about llama 2")

2024-05-02 12:09:38 INFO semantic_router.utils.logger Document 1 length: 40
2024-05-02 12:09:38 INFO semantic_router.utils.logger Document 1 trunc length: 40


RouteChoice(name=None, function_call=None, similarity_score=None)

We can also retrieve multiple routes with its associated score:

In [10]:
rl.retrieve_multiple_routes("Hi! How are you doing in politics??")

2024-05-02 12:09:40 INFO semantic_router.utils.logger Document 1 length: 35
2024-05-02 12:09:40 INFO semantic_router.utils.logger Document 1 trunc length: 35


[RouteChoice(name='politics', function_call=None, similarity_score=0.8596186767854487),
 RouteChoice(name='chitchat', function_call=None, similarity_score=0.8356239688161808)]

In [11]:
rl.retrieve_multiple_routes("I'm interested in learning about llama 2")

2024-05-02 12:09:42 INFO semantic_router.utils.logger Document 1 length: 40
2024-05-02 12:09:42 INFO semantic_router.utils.logger Document 1 trunc length: 40


[]